In [1]:
## Build an app to identify the presence of human hand!!
# Outline its coordinates and locate the fingertips as well.
## Building an hand writing app!
## First we will segment the hand out fram input video..
import cv2
import math
import numpy as np
def capture_histogram(source):
    cap = cv2.VideoCapture(source)
    while True:
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)
        # frame = cv2.resize(frame, (1000, 600))
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, 'place region of hand inside the box and press a', (5, 50), font, 0.7,  (255,255,255), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (500,100), (580, 180), (105,105,105), 2)
        box = frame[105:175, 505:575]
        
        cv2.imshow('capture histogram', frame)
        k = cv2.waitKey(10)
        if k == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            break
            
        if k == ord('a'):
            object_color = box
            cv2.destroyAllWindows()
            cap.release()
            break
    ## Because it is easier to calculate histogram of HSV.
    obj_color_hsv = cv2.cvtColor(object_color, cv2.COLOR_BGR2HSV)
    obj_hist = cv2.calcHist([obj_color_hsv], [0, 1], None, [12, 15], [0, 180, 0, 256])
        # [0, 1] means we are computing histogram for 2 channels 0th and 1st..
        ## None because we are not using a mask..
        # [12, 15] denotes the no. of bins in the corresponding channel..
        # [0, 180, 0, 256] denote the range of each channel..
    cv2.normalize(obj_hist,obj_hist, 0, 255, cv2.NORM_MINMAX)
    return obj_hist

In [2]:
## Implementing backproj. on input video feed..
def locate_object(frame, obj_hist):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV) ## Image in which we want to compute the backprojection.
    ## Apply back_projection using obj_hist as model histogram.
    obj_seg = cv2.calcBackProject([hsv_frame], [0,1], obj_hist, [0, 180, 0, 255], 1)
    ## 1 is just a scaling factor.
    _, seg_thresh = cv2.threshold(obj_seg, 70, 255, cv2.THRESH_BINARY) #This is unenhanced version of the original image. 
    
    ## applying some image operations to enhance the image..
    kernel = None
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15,15)) ## To get a circular filter..
    filtered = cv2.filter2D(seg_thresh, -1, disc) ## Applying that disc kernel over the segmented image.
    # So when we apply such a filter it tends to smoothen out the image and makes the image easier to visualise..
    eroded = cv2.erode(filtered, kernel, iterations = 2)
    dilated = cv2.dilate(filtered, kernel, iterations = 2)
    closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel) ## Closing is dilation followed by erosion.
    ## And this is a black and white enhanced image of our object..
    
    ## Masking 
    masked = cv2.bitwise_and(frame, frame, mask = closing)
    return closing, masked, seg_thresh

In [3]:
def detect_hand(frame, hist):
    return_val = {}
    detected_hand, masked, raw = locate_object(frame, hist)
    return_val['binary'] = detected_hand
    return_val['masked'] = masked
    return_val['raw'] = raw
    
    ## Now we will also find the boundaries of the image.
    image, cont, _ = cv2.findContours(detected_hand, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # The last two params are commonly used to find the boundaries or contours.
    
    palm_area = 0
    flag_area = None
    cnt = None
    
    for (i,c) in enumerate(cont):
        area = cv2.contourArea(c)
        if area > palm_area:
            palm_area = area
            flag = i
    ## This was just to find the biggest contour.        
    if flag is not None and palm_area > 10000: # if flag is not none means we have found some boundary.
        cnt = cont[flag]
        return_val['contour'] = cnt
        cpy = frame.copy()
        cv2.drawContours(cpy, [cnt], 0, (0, 255, 0), 2) ## third arg. is the contour index..
        return_val['boundaries'] = cpy
        return True, return_val
    else:
        return False, return_val

In [4]:
def dist(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

In [5]:
def filter_points(points, filter_val):
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            if points[i] and points[j] and dist(points[i], points[j]) < filter_val:
                points[j] = None
    filtered = []            
    for point in points:
        if point is not None:
            filtered.append(point)
    return filtered        

In [6]:
def extract_fingertips(hand):
    cnt = hand['contour']
    pts = []
    hull = cv2.convexHull(cnt, returnPoints = False) #False because we want to compute the indices of defects of HULL
    defects = cv2.convexityDefects(cnt, hull)
    
    ## Getting all the end points using defects and contours.
    for i in range(defects.shape[0]): ## Iterating over the rows.
        s,e,f,d = defects[i,0] # start pt, end pt, far pt, distance to farthest point..
        end = tuple(cnt[e][0])
        pts.append(end)
        
    ## Filter out the points which are too close to each other.
    filtered = filter_points(pts, 50) ## 50 is the minimum distance between two points..
    
    ## sort the fingertips in order of increasing y-coordinate..
    filtered.sort(key = lambda y: y[1])
    
    return [pt for idx, pt in zip(range(5), filtered)]

In [7]:
def plot(frame, points):
    radius = 5
    color = (0,0,255)
    thickness = -1
    for point in points:
        cv2.circle(frame, point, radius, color, thickness)

In [4]:
## Saving the model histogram for further use.
hist = capture_histogram(0)
np.save('hist.npy', hist)

In [11]:
## Now puttig it all together.
# Testing it for a test image,
import cv2

cap = cv2.VideoCapture(0)

## Initializing a black canvas. on the top of which we will draw something..
screen = np.zeros((600, 1000))

hist = np.load('hist.npy')

curr = None
prev = None ## These two variables will keep track of our fingertips

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (1000, 600))

    bool_hand, hand = detect_hand(frame, hist) # bool_hand i.e whether we got desired contour/boundary.   
    if bool_hand:
        hand_image = hand['boundaries']
        #cv2.imshow('Hand_detector', hand['boundaries'])
        fingertips = extract_fingertips(hand)
        plot(hand_image, fingertips)
        #cv2.imshow('enhanced binary', hand['binary'])
        #cv2.imshow('masked', hand['masked'])
        #cv2.imshow('raw', hand['raw'])
        prev = curr
        curr = fingertips[0]
        
        if prev and curr:
            cv2.line(screen, prev, curr, (255,0,0), 5)
        cv2.imshow('hand_detected', hand_image)
        cv2.imshow('draw', screen)    
        
    else:
        cv2.imshow('frame', frame)
        
    k = cv2.waitKey(10)
    if k == ord('q'):
        cap.release()
        break
    
cv2.destroyAllWindows()